In [1]:
import datetime as dt
import pandas as pd
import re
from pathlib import Path
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.remote.errorhandler import NoSuchElementException
from ariba_driver import Ariba
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager

DOWNLOAD_DIRECTORY = Path('/home/alex/Downloads')
REPO_DIRECTORY = '/home/alex/repos/toronto-bids/scrapers/ariba'

In [2]:
driver = Ariba(service=ChromeService(ChromeDriverManager().install()))

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6.83M/6.83M [00:00<00:00, 101MB/s]


In [3]:
driver.get("https://service.ariba.com/Discovery.aw/ad/profile?key=AN01050912625#b0")
sleep(5)
driver.find_element(By.CSS_SELECTOR, ".sap-icon--log").click()
sleep(5)
with open('username.key','r') as f:
    driver.find_element(By.NAME, "UserName").send_keys(f.read())
with open('password.key','r') as f:
    driver.find_element(By.NAME, "Password").send_keys(f.read())
driver.find_element(By.NAME, "Password").send_keys(Keys.ENTER)
sleep(5)
driver.get("https://service.ariba.com/Discovery.aw/ad/profile?key=AN01050912625#b0")

In [4]:
clicked = set()

if Path('data_summary.csv').exists():
    df = pd.read_csv('data_summary.csv')
    for title in df.title.values:
        if title == title:
            title = title.replace('...', '')
            clicked.add(title[0:100])

In [5]:
has_clicked = False

In [6]:
def wait_for_download(command, max_wait=1200):
    initial_length = len(list(DOWNLOAD_DIRECTORY.iterdir()))
    command()
    total_wait = 0
    while len(list(DOWNLOAD_DIRECTORY.iterdir())) == initial_length:
        sleep(15)
        total_wait += 15
        if total_wait > max_wait:
            return False
    return True

In [7]:
def count_directory_files(root: Path):
    if not root.exists():
        return 0
    return len(list(root.iterdir()))

In [8]:
def main_loop(has_clicked=False):
    while not has_clicked:
        elements = driver.find_elements(By.CLASS_NAME, 'ADTableBodyWhite')
        elements += driver.find_elements(By.CLASS_NAME, 'ADHiliteBlock')
        for element in elements:
            try:
                title = element.find_element(By.CLASS_NAME, 'QuoteSearchResultTitle')
            except NoSuchElementException:
                continue
            title_text = title.text
            if title_text[0:100] in clicked:
                continue
            
            print(f'Accessing {title.text}')
            try:
                date = element.find_elements(By.CLASS_NAME, 'paddingRight5')[2].text
                request_expired = dt.datetime.strptime(date[:-4], '%d %b %Y %I:%M %p') < dt.datetime.now()
                print(f'\tdate: {date}')

            except IndexError:
                request_expired = True
                print('\tNo date found')

            clicked.add(title_text[0:100])
            title.click()
            has_clicked = True

            document_id = driver.patiently_find_regex('(Doc\d{10})')
            print(f'\tDocument id is {document_id}')

            noip = driver.find_elements(By.XPATH, '//a[contains(text(),".pdf")]')
            for link in noip:
                print(f'\tPDF found, downloading {link.text}')
                wait_for_download(lambda: link.click())

            
            html_exists = Path(f'{REPO_DIRECTORY}/data/{document_id}.html').exists() or Path(
                f'{REPO_DIRECTORY}/data/{document_id}/{document_id}.html'
            ).exists()
            zip_exists = Path(f'{REPO_DIRECTORY}/data/{document_id}.zip').exists() or count_directory_files(Path(
                f'{REPO_DIRECTORY}/data/{document_id}'
            )) > 1
            
            print('\tHTML exists' if html_exists else '\tHTML does not exist')

            if zip_exists:
                print('\tZip exists')
            elif not request_expired:
                print('\tZip does not exist')
            else:
                print('\tZip does not exist, but RFP is expired')

            if not html_exists:
                with open(f'{REPO_DIRECTORY}/data/{document_id}.html', 'w') as f:
                    f.write(driver.page_source)
            if (not zip_exists) and (not request_expired):
                driver.patiently_click('//*[@id="_hfdr9c"]')  #respond to posting
                driver.patiently_click('//*[@id="_xjqay"]')  #download content
                driver.patiently_click('//*[@id="_hgesab"]', wait_after=15)  #click download attachment
                driver.patiently_click('//*[@id="_h_l$m"]/span/div/label', wait_after=5)  #click select all
                wait_for_download(
                    lambda: driver.patiently_click('//*[@id="_5wq_j"]')
                )  #download attachments (for real)
            driver.get("https://service.ariba.com/Discovery.aw/ad/profile?key=AN01050912625#b0")
            sleep(2)
            break
        if not has_clicked:
            driver.patiently_click('//*[@id="next"]', wait_after=5)

In [ ]:
while True:
    try:
        main_loop()
    except Exception as e:
        print(e)
        sleep(60 * 10)
        driver.get("https://service.ariba.com/Discovery.aw/ad/profile?key=AN01050912625#b0")

Accessing RFT for  Ashbridges Bay Treatment Plant Boiler Demolition and Installation Contract
	date: 9 May 2022 8:59 PM PDT
	Document id is Doc3374301730
	HTML exists
	Zip does not exist, but RFP is expired
Accessing RFP for Actuarial Valuation for the Toronto Fire Department Superannuation and Benefit  Fund
	date: 29 Jul 2022 9:03 AM PDT
	Document id is Doc3313053012
	HTML exists
	Zip does not exist, but RFP is expired
Accessing Doc3528836035 -  RFQ - Thermal Imaging Cameras and Related Accessories for the City of Toronto’s Fire Services Division
	date: 20 Jul 2022 8:59 PM PDT
	Document id is Doc3528836035
	HTML exists
	Zip does not exist, but RFP is expired
Accessing Request for Quotations for Power Pressure Cleaning Services for Kitchen, Dishwasher and Laundry Exhaust and Associated Ducts
	date: 14 Mar 2023 9:00 AM PDT
	Document id is Doc3853643516
	HTML exists
	Zip exists
Accessing Request For Supplier Qualification for Design Builders for retrofit projects  with the RHI program
	d